In [2]:
import pandas as pd

In [3]:
dataset_name_list = [
'CWA-0',
'CWA-1',
'CWA-2',
'CWA-3',
'CWA-4',
'CWA-5',
]
dataset_path_list = [
 './Standard_llama2-7b_42_proofwriter_6000_CWA_REAL_depth-0_1000.jsonl_01_29_22_13_09_res.csv',
 './Standard_llama2-7b_42_proofwriter_6000_CWA_REAL_depth-1_1000.jsonl_01_29_22_33_11_res.csv',
 './Standard_llama2-7b_42_proofwriter_6000_CWA_REAL_depth-2_1000.jsonl_01_29_22_53_46_res.csv',
 './Standard_llama2-7b_42_proofwriter_6000_CWA_REAL_depth-3_1000.jsonl_01_29_23_16_48_res.csv',
 './Standard_llama2-7b_42_proofwriter_6000_CWA_REAL_depth-4_1000.jsonl_01_29_23_38_20_res.csv',
 './Standard_llama2-7b_42_proofwriter_6000_CWA_REAL_depth-5_1000.jsonl_01_31_15_48_05_res.csv']

In [4]:
output_df   = pd.read_csv(dataset_path_list[0])
output_df.columns

Index(['Unnamed: 0', 'nr', 'q_nr', 'theory_nr', 'theory', 'question', 'answer',
       'meta_folder', 'meta_file', 'meta_depth', 'meta_id', 'D_maxD',
       'D_NFact', 'D_NRule', 'Q_Dep', 'Q_Len', 'Q_strategy', 'Q_proofs',
       'output_prompt', 'output_answer', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25'],
      dtype='object')

In [5]:
def parse_label(text):
    text = text.lower()
    if 'true' in text and 'false' not in text:
        return True
    elif 'true' not in text and 'false' in text:
        return False
    else:
        return 'error'
    

In [6]:
def analyse(dataset_name_list, dataset_path_list, answer_column:str):
    print(f"name\tall\t\
co:a\tco:T%\tco:PT\tco:PF\tco:RT\tco:RF\tError")  
    for dataset_name, dataset_path in zip(dataset_name_list, dataset_path_list):
        output_df   = pd.read_csv(dataset_path)
        num_all     = len(output_df)
        output_df['answer_code'] = output_df[answer_column].map(parse_label)
        output_df['correct'] = output_df['answer_code'] == output_df['answer']


        # num_answer_correct      = output_df['correct'].value_counts()[1]
        num_answer_correct      = len(output_df[output_df['correct']==True])
        num_code_acc                     = num_answer_correct / num_all
        num_code_acc                     = round(num_code_acc, 2) * 100

        # How many True
        num_code_T   = len(output_df[output_df['answer_code']==True]) /num_all
        num_code_E   = len(output_df[output_df['answer_code']=='error']) /num_all


        # How many
        num_code_RealT_PredT = len(output_df[(output_df['answer']==True) & (output_df['answer_code']==True)])
        num_code_RealT_PredF = len(output_df[(output_df['answer']==True) & (output_df['answer_code']==False)])
        num_code_RealF_PredF = len(output_df[(output_df['answer']==False) & (output_df['answer_code']==False)])
        num_code_RealF_PredT = len(output_df[(output_df['answer']==False) & (output_df['answer_code']==True)])
        # precise: in all found True, how many are real True
        # recall: in all real True, how many are cound
        # Code
        precise_T_code  = num_code_RealT_PredT / (len(output_df[output_df['answer_code']==True])+0.0000001)
        precise_F_code  = num_code_RealF_PredF  / (len(output_df[output_df['answer_code']==False])+0.0000001)
        recall_T_code   = num_code_RealT_PredT / (len(output_df[output_df['answer']==True]))
        recall_F_code   = num_code_RealF_PredF / (len(output_df[output_df['answer']==False]))
       

        # num_code_acc    = str(num_code_acc  *100)[:6]
        num_code_T      = str(num_code_T    *100)[:6]
        num_code_E      = str(num_code_E    *100)[:6]
        precise_T_code  = str(precise_T_code*100)[:6]
        precise_F_code  = str(precise_F_code  *100)[:6]
        recall_T_code   = str(recall_T_code  *100)[:6]
        recall_F_code   = str(recall_F_code  *100)[:6]
        
        # print(f"{dataset_name}\t{num_all}\t{num_answer_correct}\t{round(acc, 2)}%\t{num_answer_error}\t{error}%\t{acc_T}%\t{acc_F}%")
        print(f"{dataset_name}\t{num_all}\t\
{num_code_acc}\t{num_code_T}\t{precise_T_code}\t{precise_F_code}\t{recall_T_code}\t{recall_F_code}\t{num_code_E}")
        


# Standard 7B

In [7]:
analyse(dataset_name_list=dataset_name_list, dataset_path_list=dataset_path_list, answer_column='4')

name	all	co:a	co:T%	co:PT	co:PF	co:RT	co:RF	Error
CWA-0	1000	56.00000000000001	77.0	58.311	70.303	89.8	23.200	6.5
CWA-1	1000	50.0	79.0	53.164	76.999	84.0	15.4	11.0
CWA-2	1000	51.0	77.5	54.709	71.551	84.8	16.6	10.9
CWA-3	1000	50.0	80.2	53.990	69.148	86.6	13.0	10.4
CWA-4	1000	54.0	75.8	56.068	73.584	85.0	23.400	8.3
CWA-5	1000	49.0	69.1	54.992	65.714	76.0	23.0	13.4


# Standard 13B

In [9]:
dataset_name_list = [
'CWA-0',
'CWA-1',
'CWA-2',
'CWA-3',
'CWA-4',
'CWA-5',
]
dataset_path_list = [
 './Standard_llama-13b-hf_42_proofwriter_6000_CWA_REAL_depth-0_1000.jsonl_02_03_12_35_17_res.csv',
 './Standard_llama-13b-hf_42_proofwriter_6000_CWA_REAL_depth-1_1000.jsonl_02_03_13_18_25_res.csv',
 './Standard_llama-13b-hf_42_proofwriter_6000_CWA_REAL_depth-2_1000.jsonl_02_04_03_49_29_res.csv',
 './Standard_llama-13b-hf_42_proofwriter_6000_CWA_REAL_depth-3_1000.jsonl_02_04_03_58_11_res.csv',
 './Standard_llama-13b-hf_42_proofwriter_6000_CWA_REAL_depth-4_1000.jsonl_02_04_04_21_56_res.csv',
 './Standard_llama-13b-hf_42_proofwriter_6000_CWA_REAL_depth-5_1000.jsonl_02_04_04_36_23_res.csv']

In [10]:
analyse(dataset_name_list=dataset_name_list, dataset_path_list=dataset_path_list, answer_column='4')

name	all	co:a	co:T%	co:PT	co:PF	co:RT	co:RF	Error
CWA-0	1000	34.0	65.5	52.366	99.999	68.600	0.2	34.4
CWA-1	1000	30.0	60.6	49.504	0.0	60.0	0.0	39.4
CWA-2	1000	33.0	63.4	52.523	0.0	66.600	0.0	36.6
CWA-3	1000	32.0	65.3	49.617	0.0	64.8	0.0	34.699
CWA-4	1000	38.0	75.5	49.801	0.0	75.2	0.0	24.5
CWA-5	1000	37.0	74.0	50.270	0.0	74.4	0.0	26.0
